第一步需要导入图片，并转换为可识别的对象，即二维数组
使用Pillow导入图片，转换为黑白图像（彩色或者灰度图像可能需要前处理）
直接转换为`np.array`对象

In [ ]:
from PIL import Image
import numpy as np

image = Image.open('datas/X.png')
# convert('1')可将图像转化为二值图像
image = image.convert('1')
# 转为numpy array
array = np.array(image)

print(array)
print(f"{array.size = }")

这里的输出结果可以看到，16×16的图像已经被转换为了16×16的数组。
数组元素是True和False，没关系，这不影响后续操作。

卷积，是两个函数（信号）合并的一种方式，它可以通过一个函数对另一个函数进行“滤波”，以达到提取特征的目的。

[视频资料-卷积](https://www.youtube.com/watch?app=desktop&v=KuXjwB4LzSA)

对于此程序，我们使用3×3的卷积核进行滤波。

- 为什么采用3×3的卷积核？
    - 3×3或5×5的卷积核在CNN的实际应用中较为常见，两个3×3卷积核只需要18个权重就可以有很好的覆盖效果，提高参数的平均效益。
    - 较小的卷积核不容易忽略边缘的信号。
    - 因此通常采用多个小尺寸卷积核构造多个激活函数

In [ ]:
from scipy.signal import convolve2d

# 左下-右上卷积核
# 采用3*3卷积核，卷积核采用(-1, 2)二值
# 如果采用(0,1)那么对比度边缘（快速变化的区域）特征会被忽视
kernel_vy = np.array([[-1, -1, 2],
                      [-1, 2, -1],
                      [2, -1, -1]])
# 左上-右下卷积核
kernel_vx = np.array([[2, -1, -1],
                      [-1, 2, -1],
                      [-1, -1, 2]])
convolved_vx = convolve2d(array, kernel_vx, mode='valid')
convolved_vy = convolve2d(array, kernel_vy, mode='valid')

# 卷积结果
# 由于边缘覆盖问题，16*16的图经过3*3卷积核的采样之后，特征图的大小是14*14
print("左上-右下卷积结果:")
print(convolved_vx)
print("左下-右上卷积结果:")
print(convolved_vy)

由于滤波器从(2, 2)坐标开始平移，因此实际的卷积得到的特征图大小是14×14

池化操作

其目的是降低参数量，降低最后的向量维度，降低细微变化对整体的影响。

具体操作类似于图像转化为分辨率更低的图像。即将数组均分为多个n×n的小数组，
把这些小数组取平均值（平均池化）或取最大值（最大池化）后，按照原来的位置，填充得到新数组。

在这里，我们输入的是14×14大小图片，池化窗口的大小为2×2，得到的特征图大小是7×7。

In [ ]:
from skimage.measure import block_reduce

# 池化操作
# 在这里我们选择最大池化（在一个池化窗口中选取最大值作为池化结果）
# 14*14大小图片，以(2,2)池化的结果，得到的特征图大小是7*7
pool_size = (2, 2)
pooled_vx = block_reduce(convolved_vx, pool_size, np.max)
pooled_vy = block_reduce(convolved_vy, pool_size, np.max)

print("池化结果")
print(pooled_vx)
print(pooled_vy)

至此已经完成了卷积和池化，我们获得了两个池化后的特征图，它们已经具有了较好的泛化能力。

接下来，我们直接将两个特征图统一提取出特征向量。

In [ ]:
# 将池化后的特征图展平成特征向量，这一操作可以直接使用numpy提供的flatten()方法完成
# 实际上转换为全连接层的输入还有其他方法，这里不训练模型，因此直接连接成一个向量
flattened_vx = pooled_vx.flatten()
flattened_vy = pooled_vy.flatten()
# 将两个方向的向量拼合，可以得到一个长向量，作为该图片的特征向量
# 特征向量的长度应当是2*7*7=98
feature_vector = np.concatenate((flattened_vx, flattened_vy))
print("特征向量:")
print(feature_vector)